# Download the titanic data set from Kaggle site

### install python-dotenv

In [2]:
#!pip3 install python-dotenv

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
from dotenv import load_dotenv, find_dotenv

In [4]:
#### find the .env file from the directories

dotenv_path = find_dotenv()

##### load the env file 
load_dotenv(dotenv_path)

True

In [7]:
### extarct env variable using os path

import os

username = os.environ.get("KAGGLE_USERNAME")

username

'vipinparambil'

### Read data from the Kaggle site

In [22]:
import requests
from requests import session


In [77]:
### post pay load

payload ={
    "action":"login",
    "username": os.environ.get("KAGGLE_USERNAME"),
    "password": os.environ.get("KAGGLE_PASSWORD")    
}


### url for test.csv
url = "https://www.kaggle.com/c/titanic/download/train.csv"

### create a sesion 
with session() as c:
    c.post("https://www.kaggle.com/account/login", data=payload)
    response = c.get(url)
    print(response.headers)
    

{'Date': 'Fri, 12 Jul 2019 15:03:56 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Cache-Control': 'no-cache, no-store', 'Pragma': 'no-cache', 'Content-Encoding': 'gzip', 'Set-Cookie': 'CSRF-TOKEN=CfDJ8LdUzqlsSWBPr4Ce3rb9VL-EzvcktudzQErk_nQPvrroRMsxBpRUjKhK8rXonONRjZCSqYtSJoeL0fF3fgNIZBqO1kpyilg2s-bvvBvoocf9OB-Z7_MBheariT3zUiN1Eb1bC-OaDIiDPkfJc_6zceg; path=/; secure; httponly, XSRF-TOKEN=CfDJ8LdUzqlsSWBPr4Ce3rb9VL9XFWLWf4Z2w9_GHF5SZrB7iU7Z57z0fLtKMmkrgi0NsG4GyKH9po6E7-1-G9B7h0vzOYgfLaFt1331sGmKmNhGXWo2FK0wOw-_hR9G9r8SeKXgbErtm8ghcbmUd-30-Fk; path=/, .AspNetCore.Mvc.CookieTempDataProvider=; expires=Thu, 01 Jan 1970 00:00:00 GMT; path=/; samesite=lax', 'Vary': 'Accept-Encoding', 'Turbolinks-Location': 'https://www.kaggle.com/account/login?ReturnUrl=%2Fc%2Ftitanic%2Fdownload%2Ftrain.csv', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'Via': '1.1 google', 'Alt-Svc': 'clear', 'Transfer-Encoding': 'chunked'}


In [45]:
### Read Data and write file

import requests 

from requests import session

payload ={
    "action":"login",
    "username": os.environ.get("KAGGLE_USERNAME"),
    "password": os.environ.get("KAGGLE_PASSWORD")
}


def extract_data(url, file_path):
    #setup session
    with session() as c:
        c.post("https://www.kaggle.com/account/login", data = payload)
        with open(file_path,'w') as f:
            result = c.get(url, stream=True) ## read in stream line
            for data in result.iter_content(1024):
                f.write(data.decode('utf-8'))

    


In [46]:
## URLS
train_url = "https://www.kaggle.com/c/titanic/download/train.csv"
test_url = "https://www.kaggle.com/c/titanic/download/test.csv"

In [47]:
### Raw folder path in project data folder data/raw

##file paths

raw_data_path = os.path.join(os.path.pardir, "data", "raw")
tarin_csv_path = os.path.join(raw_data_path, "train.csv")
test_csv_path = os.path.join(raw_data_path, "test.csv")

### dwonload an write the file

extract_data(train_url, tarin_csv_path)
extract_data(test_url, test_csv_path)

In [48]:
###list files
!ls -l ../data/raw

total 24
-rw-r--r-- 1 50520484 1049089 10191 Jul 12 19:46 test.csv
-rw-r--r-- 1 50520484 1049089 10191 Jul 12 19:46 train.csv


# Build the functionality as a Script

In [66]:
### creating the script file

get_raw_data_script = os.path.join(os.path.pardir, "src", "data", "get_titanic_raw_data_script.py")

In [67]:
%%writefile $get_raw_data_script

## write the script to the file
##Import
import os
from dotenv import load_dotenv, find_dotenv
import requests
from requests import session
import logging


###Pay load
payload ={
    "action":"login",
    "username": os.environ.get("KAGGLE_USERNAME"),
    "password": os.environ.get("KAGGLE_PASSWORD")
}

def extract_data(url, file_path):
    """
    Method to extract data from kaggle site
    """
    with session() as c:
        c.post("https://www.kaggle.com/account/login", data=payload)
        with open(file_path, 'w') as f:
            response = c.get(url, stream = True)
            for data in response.iter_content(1024):
                f.write(data.decode("utf-8"))


                
### Main method

def main(project_dir):
    """
     Main method
    """
    logger = logging.getLogger(__name__)
    logger.info("Downloading the raw data ..............")
    
    #urls
    ## URLS
    train_url = "https://www.kaggle.com/c/titanic/download/train.csv"
    test_url = "https://www.kaggle.com/c/titanic/download/test.csv"
    
    ## Csv file paths    
    raw_data_path = os.path.join(project_dir, "data", "raw")
    train_csv = os.path.join(raw_data_path, "train.csv")
    test_csv = os.path.join(raw_data_path, "test.csv")
    
    ## extract data
    extract_data(train_url, train_csv)
    extract_data(test_url, test_csv)
    logger.info("Downloaded raw data files ")

    
## Cal the main

if __name__ == '__main__':
    ##get the root directory    
    project_dir = os.path.join(os.path.dirname(__file__), os.pardir, os.pardir)
    
    ## setup logger
    log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_fmt)
    
    ###load the env variable
    dot_env_path = find_dotenv()
    load_dotenv(dot_env_path)
    
    ##call main
    main(project_dir)
    


Overwriting ..\src\data\get_titanic_raw_data_script.py


In [68]:
!python $get_raw_data_script

2019-07-12 20:23:17,874 - __main__ - INFO - Downloading the raw data ..............
2019-07-12 20:23:21,974 - __main__ - INFO - Downloaded raw data files 
